In [1]:
import pandas as pd
img_train = pd.read_csv('img_train.csv')
demo_viome_train = pd.read_csv('demo_viome_train.csv')
cgm_train = pd.read_csv('cgm_train.csv')
img_train = img_train.sort_values(by=['Subject ID', 'Day'],ascending=True)
demo_viome_train = demo_viome_train.sort_values(by=['Subject ID'],ascending=True)
cgm_train = cgm_train.sort_values(by=['Subject ID', 'Day'],ascending=True)

In [3]:
demo_viome_train=pd.merge(img_train, demo_viome_train, on="Subject ID", how="left")
demo_viome_train=demo_viome_train.drop(columns=['Image Before Breakfast', 'Image Before Lunch'])
demo_viome_train.head()

,Subject ID,Day,Age,Gender,Weight,Height,Race,Diabetes Status,A1C,Baseline Fasting Glucose,...,Triglycerides,Cholesterol,HDL,Non-HDL,LDL,VLDL,CHO/HDL Ratio,HOMA-IR,BMI,Viome
0,1,2,27,0,133.8,65.0,Hispanic/Latino,1,5.4,91.0,...,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,"-11.691621427726268,-7.744787588227839,-7.6142..."
1,1,3,27,0,133.8,65.0,Hispanic/Latino,1,5.4,91.0,...,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,"-11.691621427726268,-7.744787588227839,-7.6142..."
2,1,4,27,0,133.8,65.0,Hispanic/Latino,1,5.4,91.0,...,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,"-11.691621427726268,-7.744787588227839,-7.6142..."
3,1,5,27,0,133.8,65.0,Hispanic/Latino,1,5.4,91.0,...,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,"-11.691621427726268,-7.744787588227839,-7.6142..."
4,1,6,27,0,133.8,65.0,Hispanic/Latino,1,5.4,91.0,...,67.0,216.0,74.0,142.0,130.0,13.0,2.9,0.561728,22.263053,"-11.691621427726268,-7.744787588227839,-7.6142..."


In [5]:
import numpy as np
import ast
import cv2

# Assuming img_train['Image Before Lunch'][0] is a string representing a nested list
default_lunch_image_str = img_train['Image Before Lunch'][0]  # Get the string from your data

# Step 1: Convert the string to a nested list using ast.literal_eval()
try:
    # Parse the string into a Python list
    default_lunch_image = ast.literal_eval(default_lunch_image_str)

    # Step 2: Convert the nested list to a NumPy array
    default_lunch_image = np.array(default_lunch_image)

    # Step 3: Check if the image is valid (a NumPy array and 3D shape)
    if isinstance(default_lunch_image, np.ndarray):
        print("Image is a valid NumPy array.")
        
        if default_lunch_image.ndim == 3:  # Should be (height, width, 3) for RGB
            print(f"Image shape: {default_lunch_image.shape}")  # Should be (height, width, 3)

            # Step 4: Resize the image
            default_lunch_image = default_lunch_image.astype(np.uint8)
            default_lunch_image_resized = cv2.resize(default_lunch_image, (224, 224))
            print("Image resized successfully.")
        else:
            print("Error: Image does not have 3 dimensions (height, width, channels).")
    else:
        print("Error: The image is not a valid NumPy array.")

except Exception as e:
    print(f"Error parsing image data: {e}")
    # Optionally replace with a default fallback image if needed


Image is a valid NumPy array.
Image shape: (64, 64, 3)
Image resized successfully.


In [7]:
import numpy as np
import ast
from tensorflow.keras.preprocessing import image
import cv2

# Resize images to a consistent shape (e.g., 224x224 for CNN models)
def preprocess_images(image_list):
    processed_images = []

    for img_str in image_list:
        try:
            # Convert the string to a nested list, then to a NumPy array
            img = ast.literal_eval(img_str)
            img = np.array(img)

            # Check if the image has the correct shape (64, 64, 3)
            if img.shape == (64, 64, 3):
                # Resize the valid image to 224x224
                img = img.astype(np.uint8)
                img_resized = cv2.resize(img, (224, 224))
            else:
                # If invalid, use the default resized image
                img_resized = default_lunch_image_resized
        except Exception as e:
            # If there's an error in parsing, use the default resized image
            print(f"Error processing image: {e}")
            img_resized = default_lunch_image_resized

        # Append the processed (or default) image to the list
        processed_images.append(img_resized)

    return np.array(processed_images)

# Assuming 'img_train' contains a column 'Image Before Lunch' with string representations of images
lunch_images = preprocess_images(img_train['Image Before Lunch'].tolist())

# Check the shape of the processed images (optional)
print(f"Processed images shape: {lunch_images.shape}")


Processed images shape: (324, 224, 224, 3)


In [9]:
import ast

# Function to process CGM data into usable format
def process_cgm_data(cgm_str):
    cgm_list = ast.literal_eval(cgm_str)  # Converts the string to a list of tuples
    # Extract glucose values (ignore timestamps for now)
    glucose_values = [entry[1] for entry in cgm_list]
    return glucose_values

# Apply to the CGM data column
cgm_train['Processed CGM'] = cgm_train['CGM Data'].apply(process_cgm_data)
cgm_train['Processed CGM'] = cgm_train['Processed CGM'].apply(np.array)
print(cgm_train['Processed CGM'][0].dtype)


float64


In [11]:
from sklearn.preprocessing import StandardScaler

# Normalize numeric features for the model
scaler = StandardScaler()
biometric_features = demo_viome_train.drop(['Subject ID','Day', 'Viome', 'Race'], axis=1)  # Drop Subject ID
biometric_features_normalized = scaler.fit_transform(biometric_features)


In [13]:
label_train = pd.read_csv('label_train.csv')
label_train = label_train.sort_values(by=['Subject ID', 'Day'],ascending=True)

In [25]:
import pandas as pd

# Example DataFrame (replace with your actual data)
data = pd.merge(biometric_features, label_train['Lunch Calories'], how='inner', left_index=True, right_index=True)

df = pd.DataFrame(data)

# Compute correlation matrix
correlation_matrix = df.corr()

# Extract the correlation of each feature with the label
correlation_with_label = correlation_matrix['Lunch Calories'].drop('Lunch Calories')  # Drop label itself to avoid self-correlation

# Convert to a DataFrame for better visualization
correlation_table = pd.DataFrame(correlation_with_label).reset_index()
correlation_table.columns = ['Feature', 'Correlation with Lunch Calories']

# Display the correlation table
print(correlation_table)


                     Feature  Correlation with Lunch Calories
0                        Age                     1.660607e-17
1                     Gender                    -5.849520e-17
2                     Weight                    -8.981125e-17
3                     Height                     7.888936e-17
4            Diabetes Status                    -6.512129e-17
5                        A1C                    -1.233887e-18
6   Baseline Fasting Glucose                    -3.636025e-17
7                    Insulin                    -1.136738e-16
8              Triglycerides                    -1.048787e-16
9                Cholesterol                     1.862382e-16
10                       HDL                     1.144243e-16
11                   Non-HDL                     1.440265e-16
12                       LDL                     1.870320e-16
13                      VLDL                    -2.598086e-17
14             CHO/HDL Ratio                     6.462734e-17
15      